# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "st" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [0]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-st-baseline


In [0]:
# Install opus-tools
! pip install opustools-pkg

In [0]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-st.xml.gz not found. The following files are available for downloading:

   7 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-st.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
 108 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/st.zip

 378 MB Total size
./JW300_latest_xml_en-st.xml.gz ... 100% of 7 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_st.zip ... 100% of 108 MB


In [0]:
! head -n 5 jw300.$src

Unity in Worship ​ — What Should It Mean for You ?
As we draw closer and closer to the end of this system , pressures and tensions mount .
So it is more difficult , and yet more urgent than ever , to stay on the road leading to life , a road that requires unity in worship .
The new book United in Worship of the Only True God lovingly helps us to help ourselves and others through this critical period .
Please send me , postpaid , the 192 - page , hardbound , illustrated book United in Worship of the Only True God .


In [0]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-23 07:59:12--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2020-01-23 07:59:12 (6.61 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-23 07:59:15--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-st.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 7135/927465 lines since contained in test set.


,source_sentence,target_sentence
0,Unity in Worship ​ — What Should It Mean for Y...,Ho Kopana Borapeling — Ho Lokela ho Bolela Eng...
1,As we draw closer and closer to the end of thi...,"Ha re ntse re atamela bofelo ba tsamaiso ena ,..."
2,"So it is more difficult , and yet more urgent ...","Lintho li ba boima le ho feta , ’ me leha ho l..."


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.30 0.00 percent complete
00:00:21.69 0.12 percent complete
00:00:43.18 0.24 percent complete
00:01:05.38 0.36 percent complete
00:01:27.09 0.48 percent complete
00:01:49.34 0.60 percent complete
00:02:10.64 0.72 percent complete
00:02:34.87 0.84 percent complete


00:02:56.86 0.96 percent complete
00:03:18.60 1.07 percent complete
00:03:39.93 1.19 percent complete
00:04:01.79 1.31 percent complete
00:04:23.34 1.43 percent complete
00:04:43.58 1.55 percent complete
00:05:03.72 1.67 percent complete
00:05:25.94 1.79 percent complete
00:05:45.13 1.91 percent complete
00:06:04.05 2.03 percent complete
00:06:22.94 2.15 percent complete
00:06:41.45 2.27 percent complete
00:07:00.40 2.39 percent complete
00:07:19.11 2.51 percent complete
00:07:38.01 2.63 percent complete
00:07:58.49 2.75 percent complete
00:08:18.56 2.87 percent complete
00:08:39.29 2.98 percent complete
00:09:00.46 3.10 percent complete
00:09:21.68 3.22 percent complete
00:09:44.15 3.34 percent complete
00:10:06.43 3.46 percent complete
00:10:29.21 3.58 percent complete
00:10:52.27 3.70 percent complete
00:11:13.86 3.82 percent complete
00:11:35.30 3.94 percent complete
00:11:57.55 4.06 percent complete
00:12:19.65 4.18 percent complete
00:12:41.32 4.30 percent complete
00:13:02.93 4.

00:23:47.92 7.88 percent complete
00:24:09.44 8.00 percent complete
00:24:31.61 8.12 percent complete
00:24:53.52 8.24 percent complete
00:25:14.98 8.36 percent complete
00:25:37.45 8.48 percent complete
00:26:01.03 8.60 percent complete
00:26:23.18 8.71 percent complete
00:26:44.70 8.83 percent complete
00:27:06.63 8.95 percent complete
00:27:28.53 9.07 percent complete
00:27:50.25 9.19 percent complete
00:28:11.74 9.31 percent complete
00:28:33.00 9.43 percent complete
00:28:53.66 9.55 percent complete
00:29:12.40 9.67 percent complete
00:29:31.87 9.79 percent complete
00:29:50.41 9.91 percent complete
00:30:08.87 10.03 percent complete
00:30:28.43 10.15 percent complete
00:30:47.93 10.27 percent complete
00:31:07.05 10.39 percent complete
00:31:27.66 10.51 percent complete
00:31:46.60 10.62 percent complete
00:32:06.09 10.74 percent complete
00:32:25.66 10.86 percent complete
00:32:46.61 10.98 percent complete
00:33:08.45 11.10 percent complete
00:33:29.97 11.22 percent complete
00:

00:46:23.91 15.40 percent complete
00:46:46.15 15.52 percent complete
00:47:08.22 15.64 percent complete
00:47:30.36 15.76 percent complete
00:47:52.72 15.88 percent complete
00:48:15.04 16.00 percent complete
00:48:37.87 16.12 percent complete
00:48:59.92 16.24 percent complete
00:49:22.13 16.36 percent complete
00:49:44.19 16.47 percent complete
00:50:05.34 16.59 percent complete
00:50:26.90 16.71 percent complete
00:50:49.00 16.83 percent complete
00:51:10.77 16.95 percent complete
00:51:32.95 17.07 percent complete
00:51:54.98 17.19 percent complete
00:52:18.59 17.31 percent complete


00:52:40.71 17.43 percent complete
00:53:02.29 17.55 percent complete
00:53:23.95 17.67 percent complete
00:53:46.47 17.79 percent complete
00:54:08.48 17.91 percent complete
00:54:30.33 18.03 percent complete
00:54:52.72 18.15 percent complete
00:55:17.05 18.27 percent complete
00:55:39.08 18.38 percent complete
00:56:00.76 18.50 percent complete
00:56:22.82 18.62 percent complete
00:56:44.95 18.74 percent complete
00:57:06.81 18.86 percent complete
00:57:28.90 18.98 percent complete


...

04:05:26.22 86.67 percent complete
04:05:45.78 86.79 percent complete
04:06:06.11 86.91 percent complete
04:06:25.18 87.03 percent complete
04:06:44.43 87.15 percent complete
04:07:03.68 87.27 percent complete
04:07:23.25 87.39 percent complete
04:07:42.77 87.51 percent complete
04:08:01.57 87.63 percent complete
04:08:20.77 87.75 percent complete
04:08:41.57 87.86 percent complete
04:09:00.61 87.98 percent complete
04:09:20.18 88.10 percent complete
04:09:40.53 88.22 percent complete


04:10:00.93 88.34 percent complete
04:10:21.56 88.46 percent complete
04:10:42.34 88.58 percent complete
04:11:02.69 88.70 percent complete
04:11:25.70 88.82 percent complete
04:11:47.07 88.94 percent complete
04:12:06.90 89.06 percent complete
04:12:26.67 89.18 percent complete
04:12:46.21 89.30 percent complete
04:13:05.99 89.42 percent complete
04:13:25.88 89.54 percent complete
04:13:45.80 89.66 percent complete
04:14:06.67 89.77 percent complete
04:14:26.45 89.89 percent complete
04:14:46.81 90.01 percent complete
04:15:06.71 90.13 percent complete
04:15:26.34 90.25 percent complete
04:15:46.29 90.37 percent complete
04:16:05.73 90.49 percent complete
04:16:25.47 90.61 percent complete
04:16:46.51 90.73 percent complete
04:17:05.38 90.85 percent complete
04:17:25.29 90.97 percent complete
04:17:44.40 91.09 percent complete
04:18:03.82 91.21 percent complete
04:18:23.56 91.33 percent complete
04:18:42.84 91.45 percent complete
04:19:02.21 91.57 percent complete
04:19:22.98 91.68 pe

04:25:02.19 93.71 percent complete
04:25:22.20 93.83 percent complete
04:25:41.65 93.95 percent complete
04:26:01.42 94.07 percent complete
04:26:21.07 94.19 percent complete


04:26:40.36 94.31 percent complete
04:26:59.27 94.43 percent complete
04:27:19.39 94.55 percent complete
04:27:39.37 94.67 percent complete
04:27:58.34 94.79 percent complete
04:28:18.19 94.91 percent complete
04:28:37.38 95.03 percent complete
04:28:56.39 95.15 percent complete
04:29:16.15 95.27 percent complete
04:29:35.67 95.39 percent complete
04:29:55.31 95.50 percent complete
04:30:16.26 95.62 percent complete
04:30:35.68 95.74 percent complete
04:30:55.61 95.86 percent complete
04:31:15.15 95.98 percent complete
04:31:35.05 96.10 percent complete
04:31:54.41 96.22 percent complete
04:32:13.97 96.34 percent complete
04:32:33.96 96.46 percent complete
04:32:54.95 96.58 percent complete
04:33:14.70 96.70 percent complete
04:33:34.60 96.82 percent complete
04:33:54.05 96.94 percent complete
04:34:13.07 97.06 percent complete
04:34:32.93 97.18 percent complete
04:34:52.62 97.30 percent complete
04:35:12.12 97.41 percent complete
04:35:33.47 97.53 percent complete
04:35:53.19 97.65 pe

04:40:51.99 99.44 percent complete
04:41:11.19 99.56 percent complete
04:41:31.81 99.68 percent complete
04:41:51.23 99.80 percent complete
04:42:10.38 99.92 percent complete


In [0]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
Little did I realize that in future years I would spend the major portion of my life continuing the work of these pilgrims by serving as a traveling overseer of Jehovah’s Witnesses .
These were things I never imagined I would be able to get rid of . ”
Ruins of the theater at Ephesus
We were arrested and taken to the police station .
Today , a more far - reaching destruction is looming , one that will bring an end to this entire system of things .
The Bible provides the answer when it says that humans were created “ in God’s image , ” that is , they possess the same basic godly qualities . ​ — Genesis 1 : 27 .
And whom did the king assign to sing out Mordecai’s praises ?
Later , we began pioneering together in West Berlin .
Just as he did in the past , today God is certainly able to deliver his servants from dangerous situations , and those who feel that they have benefited from his protection are not to be criticized .
As long as we are being guided by Bible principles

In [0]:
# Create directory, move everyone we care about to the google drive for training
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"